In [2]:
import pandas as pd

In [3]:
# Store filepath in a variable
data = "Resources/Airline_Delay_Cause.csv"

In [4]:
# Read Data file with the pandas library

data_df = pd.read_csv(data, encoding="ISO-8859-1")

In [5]:
# Preview the data within the Panda Dataframe

data_df.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2022,5,9E,Endeavor Air Inc.,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",136.0,7.0,5.95,0.00,...,0.0,1.00,0.0,0.0,255.0,222.0,0.0,4.0,0.0,29.0
1,2022,5,9E,Endeavor Air Inc.,ABY,"Albany, GA: Southwest Georgia Regional",91.0,16.0,7.38,0.00,...,0.0,6.09,0.0,0.0,884.0,351.0,0.0,81.0,0.0,452.0
2,2022,5,9E,Endeavor Air Inc.,ACK,"Nantucket, MA: Nantucket Memorial",19.0,2.0,0.13,0.00,...,0.0,0.88,1.0,0.0,138.0,4.0,0.0,106.0,0.0,28.0
3,2022,5,9E,Endeavor Air Inc.,AEX,"Alexandria, LA: Alexandria International",88.0,14.0,7.26,0.76,...,0.0,1.64,0.0,0.0,947.0,585.0,35.0,125.0,0.0,202.0
4,2022,5,9E,Endeavor Air Inc.,AGS,"Augusta, GA: Augusta Regional at Bush Field",181.0,19.0,13.84,0.00,...,0.0,2.09,0.0,0.0,808.0,662.0,0.0,87.0,0.0,59.0


In [6]:
# split() up the airport_name column to parse out City(s), State, and Airport name.  The State data will be used for the params in the API call. 

# new data frame with split value columns. Starting with the city name. 
new = data_df["airport_name"].str.split(",", n = 1, expand = True)
 
# making separate city column from new data frame
data_df["city"]= new[0]
 
# making separate column to parse out state from airport name
data_df["split1"]= new[1]

# Dropping old airport_name column
data_df.drop(columns =["airport_name"], inplace = True)
 


In [7]:
# Display updated dataframe with city name in it's own column. 
data_df.head(1)

,year,month,carrier,carrier_name,airport,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,...,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,city,split1
0,2022,5,9E,Endeavor Air Inc.,ABE,136.0,7.0,5.95,0.0,0.05,...,0.0,0.0,255.0,222.0,0.0,4.0,0.0,29.0,Allentown/Bethlehem/Easton,PA: Lehigh Valley International


In [8]:
# using the same split() method, we're now splitting the state and airport name.
new1 = data_df["split1"].str.split(":", n = 1, expand = True)
 
# making separate city column from new data frame
data_df["state"]= new1[0]
 
# making separate column to parse out state from airport name
data_df["airport_name"]= new1[1]

# Dropping old airport_name column
data_df.drop(columns =["split1"], inplace = True)

In [9]:
#confirm that the old airport_name column with city, state, and airport name are now in their own columns (far right columns). 
data_df.head(1)

,year,month,carrier,carrier_name,airport,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,...,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,city,state,airport_name
0,2022,5,9E,Endeavor Air Inc.,ABE,136.0,7.0,5.95,0.0,0.05,...,0.0,255.0,222.0,0.0,4.0,0.0,29.0,Allentown/Bethlehem/Easton,PA,Lehigh Valley International


In [10]:
#create a dataframe with only the airport information used for API calls.
# 1. remove any duplicates from the dataframe
# 2. Reset the index from the previous dataframe (otherwise, for example, the index number would range from 1 to 3k)
# 3. Pull only the columns we want from the previous dataframe. Airport Code, Airport Name. State. City. 
# 4. Rename columns to better describe the data

airport_df = data_df.drop_duplicates(["airport_name", "state", "city"]).reset_index(drop=True)[['airport', 'airport_name', 'state', 'city']]
airport_df = pd.DataFrame(airport_df.rename(columns={'airport':'airport_code'}))
airport_df

,airport_code,airport_name,state,city
0,ABE,Lehigh Valley International,PA,Allentown/Bethlehem/Easton
1,ABY,Southwest Georgia Regional,GA,Albany
2,ACK,Nantucket Memorial,MA,Nantucket
3,AEX,Alexandria International,LA,Alexandria
4,AGS,Augusta Regional at Bush Field,GA,Augusta
...,...,...,...,...
415,MKK,Molokai,HI,Hoolehua
416,ILE,Skylark Field,TX,Killeen
417,SKA,Fairchild AFB,WA,Spokane
418,CBM,Columbus AFB,MS,Columbus


In [45]:
#create a dataframe for monthly data enteries of weather delays by carrier and airport.
# 1. Grab only the columns of data we need.
# 2. Rename columns to better describe the data

delay_data_df = data_df[['year', 'month', 'carrier', 'carrier_name', 'airport', 'airport_name', 'arr_flights', 'weather_delay']]
delay_data_df = pd.DataFrame(delay_data_df.rename(columns={'carrier':'carrier_code', 'airport':'airport_code', 'arr_flights':'arrival_flights'}))
delay_data_df

,year,month,carrier_code,carrier_name,airport_code,airport_name,arrival_flights,weather_delay
0,2022,5,9E,Endeavor Air Inc.,ABE,Lehigh Valley International,136.0,0.0
1,2022,5,9E,Endeavor Air Inc.,ABY,Southwest Georgia Regional,91.0,0.0
2,2022,5,9E,Endeavor Air Inc.,ACK,Nantucket Memorial,19.0,0.0
3,2022,5,9E,Endeavor Air Inc.,AEX,Alexandria International,88.0,35.0
4,2022,5,9E,Endeavor Air Inc.,AGS,Augusta Regional at Bush Field,181.0,0.0
...,...,...,...,...,...,...,...,...
318012,2003,6,DL,Delta Air Lines Inc.,SEA,Seattle/Tacoma International,480.0,164.0
318013,2003,6,DL,Delta Air Lines Inc.,SFO,San Francisco International,505.0,138.0
318014,2003,6,DL,Delta Air Lines Inc.,SJC,Norman Y. Mineta San Jose International,146.0,0.0
318015,2003,6,DL,Delta Air Lines Inc.,SJU,Luis Munoz Marin International,95.0,0.0


In [21]:
#create database for carrier name and code 
carrier_df = data_df.drop_duplicates(["carrier_name", "carrier"]).reset_index(drop=True)[["carrier_name", "carrier"]]
carrier_df = pd.DataFrame(carrier_df.rename(columns={'carrier':'carrier_code'}))
carrier_df

,carrier_name,carrier_code
0,Endeavor Air Inc.,9E
1,American Airlines Inc.,AA
2,Alaska Airlines Inc.,AS
3,JetBlue Airways,B6
4,Delta Air Lines Inc.,DL
5,Frontier Airlines Inc.,F9
6,Allegiant Air,G4
7,Hawaiian Airlines Inc.,HA
8,Envoy Air,MQ
9,Spirit Air Lines,NK


In [43]:
state_df = data_df.drop_duplicates(["state"]).reset_index(drop=True)[["state"]].reset_index()
state_df.rename(columns={'index':'state_id'})

,state_id,state
0,0,PA
1,1,GA
2,2,MA
3,3,LA
4,4,NY
5,5,WI
6,6,TX
7,7,NC
8,8,MI
9,9,ME


In [26]:
#export csv file 
airport_df.to_csv('Resources/airport_df.csv', index=False)

In [32]:
#export csv file 
delay_data_df.to_csv('Resources/delay_data_df.csv', index=False)

In [33]:
#export csv file 
carrier_df.to_csv('Resources/carrier_df.csv', index=False)

In [29]:
#export csv file 
state_df.to_csv('Resources/state_df.csv', index=False)